In [2]:
import os
from glob import glob
import re
import numpy as np
import cv2
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
def compute_flow(frame1, frame2):
    # convert to grayscale
    gray1 = cv2.cvtColor(frame1, cv2.COLOR_BGR2GRAY)
    gray2 = cv2.cvtColor(frame2, cv2.COLOR_BGR2GRAY)

    # blurr image
    gray1 = cv2.GaussianBlur(gray1, dst=None, ksize=(3,3), sigmaX=5)
    gray2 = cv2.GaussianBlur(gray2, dst=None, ksize=(3,3), sigmaX=5)
    
    '''flow_params = dict(
        winSize=(5, 5),
        maxLevel=5,
        flow=None,
        flags=0
    )

    flow = cv2.calcOpticalFlowDenseRLOF(gray1, gray2, **flow_params)'''
    
    flow = cv2.calcOpticalFlowFarneback(gray1, gray2, None,
                                        pyr_scale=0.75,
                                        levels=3,
                                        winsize=7,
                                        iterations=3,
                                        poly_n=8,
                                        poly_sigma=1.2,
                                        flags=0)
    #print(flow)
    return flow


def get_flow_viz(flow):
    """ Obtains BGR image to Visualize the Optical Flow 
        """
    hsv = np.zeros((flow.shape[0], flow.shape[1], 3), dtype=np.uint8)
    hsv[..., 1] = 255

    mag, ang = cv2.cartToPolar(flow[..., 0], flow[..., 1])
    hsv[..., 0] = ang*180/np.pi/2
    hsv[..., 2] = cv2.normalize(mag, None, 0, 255, cv2.NORM_MINMAX)
    rgb = cv2.cvtColor(hsv, cv2.COLOR_HSV2RGB)
    return rgb

def get_motion_mask(flow_mag, motion_thresh=1, kernel=np.ones((7,7))):
    """ Obtains Detection Mask from Optical Flow Magnitude
        Inputs:
            flow_mag (array) Optical Flow magnitude
            motion_thresh - thresold to determine motion
            kernel - kernal for Morphological Operations
        Outputs:
            motion_mask - Binray Motion Mask
        """
    motion_mask = np.uint8(flow_mag > motion_thresh)*255
    motion_mask = cv2.erode(motion_mask, kernel, iterations=1)
    motion_mask = cv2.morphologyEx(motion_mask, cv2.MORPH_OPEN, kernel, iterations=1)
    motion_mask = cv2.morphologyEx(motion_mask, cv2.MORPH_CLOSE, kernel, iterations=3)
    
    return motion_mask

def get_contour_detections_2(fr, mask, ang, angle_thresh=2, thresh=300):
    """ Obtains initial proposed detections from contours discoverd on the
        mask. Scores are taken as the bbox area, larger is higher.
        Inputs:
            mask - thresholded image mask
            angle_thresh - threshold for flow angle standard deviation
            thresh - threshold for contour size
        Outputs:
            detectons - array of proposed detection bounding boxes and scores 
                        [[x1,y1,x2,y2,s]]
        """
    # get mask contours
    contours, _ = cv2.findContours(mask, 
                                   cv2.RETR_EXTERNAL, # cv2.RETR_TREE, 
                                   cv2.CHAIN_APPROX_TC89_L1)
    '''cv2.drawContours(fr, contours, -1, (0, 255, 0), 3) 
    plt.imshow(cv2.cvtColor(fr, cv2.COLOR_BGR2RGB))
    plt.axis('off')
    plt.show()'''
    temp_mask = np.zeros_like(mask) # used to get flow angle of contours
    angle_thresh = angle_thresh*ang.std()
    detections = []
    for cnt in contours:
        # get area of contour
        x,y,w,h = cv2.boundingRect(cnt)
        area = w*h

        # get flow angle inside of contour
        cv2.drawContours(temp_mask, [cnt], 0, (255,), -1)
        flow_angle = ang[np.nonzero(temp_mask)]

        if (area > thresh) and (flow_angle.std() < angle_thresh): # hyperparameter
            detections.append([x,y,x+w,y+h, area])

    return np.array(detections)

def draw_bboxesRecortes(frame, detections, output_folder, count):
    for i, det in enumerate(detections):
        x1, y1, x2, y2 = det

        # Recorta la región del bounding box del frame original
        roi = frame[y1:y2, x1:x2].copy()
        output_path = os.path.join(output_folder, f"recorte_{count}_{i}.png")
        cv2.imwrite(output_path, roi)
        # Dibuja el bounding box en la región recortada
        cv2.rectangle(frame, (x1,y1), (x2,y2), (0,255,0), 3)
        
def draw_bboxes(frame, detections):
    for det in detections:
        x1,y1,x2,y2 = det
        cv2.rectangle(frame, (x1,y1), (x2,y2), (0,255,0), 3)


In [ ]:
def remove_contained_bboxes(boxes):
    """ Removes all smaller boxes that are contained within larger boxes.
        Requires bboxes to be soirted by area (score)
        Inputs:
            boxes - array bounding boxes sorted (descending) by area 
                    [[x1,y1,x2,y2]]
        Outputs:
            keep - indexes of bounding boxes that are not entirely contained 
                   in another box
        """
    check_array = np.array([True, True, False, False])
    keep = list(range(0, len(boxes)))
    for i in keep: # range(0, len(bboxes)):
        for j in range(0, len(boxes)):
            # check if box j is completely contained in box i
            if np.all((np.array(boxes[j]) >= np.array(boxes[i])) == check_array):
                try:
                    keep.remove(j)
                except ValueError:
                    continue
    return keep

def non_max_suppression(boxes, scores, threshold=1e-1):
    """
    Perform non-max suppression on a set of bounding boxes 
    and corresponding scores.
    Inputs:
        boxes: a list of bounding boxes in the format [xmin, ymin, xmax, ymax]
        scores: a list of corresponding scores 
        threshold: the IoU (intersection-over-union) threshold for merging bboxes
    Outputs:
        boxes - non-max suppressed boxes
    """
    # Sort the boxes by score in descending order
    boxes = boxes[np.argsort(scores)[::-1]]

    # remove all contained bounding boxes and get ordered index
    order = remove_contained_bboxes(boxes)

    keep = []
    while order:
        i = order.pop(0)
        keep.append(i)
        for j in order:
            # Calculate the IoU between the two boxes
            intersection = max(0, min(boxes[i][2], boxes[j][2]) - max(boxes[i][0], boxes[j][0])) * \
                           max(0, min(boxes[i][3], boxes[j][3]) - max(boxes[i][1], boxes[j][1]))
            union = (boxes[i][2] - boxes[i][0]) * (boxes[i][3] - boxes[i][1]) + \
                    (boxes[j][2] - boxes[j][0]) * (boxes[j][3] - boxes[j][1]) - intersection
            iou = intersection / union

            # Remove boxes with IoU greater than the threshold
            if iou > threshold:
                order.remove(j)
                
    return boxes[keep]

In [16]:

def procesar_frames(frame1, frame2, numframes):
    fr2 = frame2.copy()
    fr2_1 = fr2.copy()
    fr2_2 = frame2.copy()
    fr1 = frame1.copy()
    flow = compute_flow(fr1, fr2)
    mag, ang = cv2.cartToPolar(flow[..., 0], flow[..., 1])
    #cmap_result = plt.get_cmap('hsv_r')(np.log(mag/mag.max()))
    rgb = get_flow_viz(flow)
    outGRAD = rgb*50
    motion_thresh = np.c_[np.linspace(0.3, 1, frame1.shape[0])].repeat(frame1.shape[1], axis=-1)
    mask = get_motion_mask(mag, motion_thresh=motion_thresh)
    mask_rgb = cv2.cvtColor(mask, cv2.COLOR_GRAY2RGB)
    detections = get_contour_detections_2(fr2_1, mask, ang, angle_thresh=2, thresh=400)
    #print(detections.shape)
    bboxes = detections[:, :4]
    outGRAD = cv2.applyColorMap(cv2.convertScaleAbs(np.log(mag/mag.max())*200), cv2.COLORMAP_TURBO)
    scores = detections[:, -1]
    for box in bboxes:
        x1,y1,x2,y2 = box
        cv2.rectangle(mask_rgb, (x1,y1), (x2,y2), (255,0,0), 3)
    #mask_rgb
    nms_bboxes = non_max_suppression(bboxes, scores, threshold=0.4)
    mask_rgb_detections = cv2.cvtColor(mask, cv2.COLOR_GRAY2RGB)
    for det in nms_bboxes:
        x1, y1, x2, y2 = det
        cv2.rectangle(mask_rgb_detections, (x1, y1), (x2, y2), (255, 0, 0), 3)
    draw_bboxesRecortes(fr2_2, nms_bboxes, 'esquina', numframes)
    return outGRAD,fr2_2

def main(video_path, output_video_path_G, output_video_path_resultado):
    numframes = 0
    cap = cv2.VideoCapture(video_path)
    # Asegúrate de que el video se haya abierto correctamente
    if not cap.isOpened():
        print("Error al abrir el video.")
        return

    # Obtiene las propiedades del video
    fps = cap.get(cv2.CAP_PROP_FPS)
    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

    # Crea un objeto VideoWriter para guardar el nuevo video
    fps_deseado = 0.8
    fourcc = cv2.VideoWriter_fourcc(*'XVID')  # or 'H264'
    out_G = cv2.VideoWriter(output_video_path_G, fourcc, fps_deseado, (frame_width, frame_height))
    out_resultado = cv2.VideoWriter(output_video_path_resultado, fourcc, fps_deseado, (frame_width, frame_height))
    # Lee el primer frame
    ret, frame1 = cap.read()

    # Establece la frecuencia deseada (un frame por segundo)
    fps_deseado = 1

    while True:
        # Lee el siguiente frame
        ret, frame2 = cap.read()
        if not ret:
            break
        # Procesa los frames
        resultadoG, resultado = procesar_frames(frame1, frame2, numframes)
        numframes = numframes +1
        out_G.write(resultadoG)
        # Guarda el resultado en el nuevo video
        out_resultado.write(resultado)
        
        # Actualiza el frame1 para el próximo ciclo
        frame1 = frame2

        # Salta frames para alcanzar la frecuencia deseada
        saltos = int(fps / fps_deseado)
        for _ in range(saltos - 1):
            cap.read()

    # Libera los objetos de captura y escritura de video
    cap.release()
    out_G.release()
    out_resultado.release()

    print(f"Video procesado y guardado en {output_video_path}.")

# Ruta del video de entrada
video_path = 'esquina.mp4'  # Reemplaza con la ruta de tu video

# Ruta del video de salida
output_video_path = 'resultadoEsquina.mp4'
output_video_path_Gradient = 'resultadoGradientEsquina.mp4'

# Llama a la función principal para procesar los frames y guardar el nuevo video
main(video_path, output_video_path_Gradient, output_video_path)

C:\Users\Daphne\AppData\Local\Temp\ipykernel_14908\672892232.py:17: RuntimeWarning: divide by zero encountered in log
  outGRAD = cv2.applyColorMap(cv2.convertScaleAbs(np.log(mag/mag.max())*200), cv2.COLORMAP_TURBO)


Video procesado y guardado en resultadoEsquina.mp4.


In [ ]:
import os
import cv2
import numpy as np

import os
import cv2
import numpy as np

def resize_and_add_margin(input_folder, output_folder, target_size, final_size):
    # Asegúrate de que la carpeta de salida exista
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    # Recorre todas las imágenes en la carpeta de entrada
    for filename in os.listdir(input_folder):
        if filename.endswith(('.jpg', '.jpeg', '.png', '.bmp')):  # Puedes agregar más extensiones según sea necesario
            input_path = os.path.join(input_folder, filename)

            # Lee la imagen
            img = cv2.imread(input_path)

            # Obtiene las dimensiones de la imagen original
            height, width, _ = img.shape

            # Calcula el tamaño de la imagen objetivo
            new_size = max(height, width)
            target_shape = (new_size, new_size)

            # Calcula la posición del área de la imagen en el lienzo negro
            y_pos = (new_size - height) // 2
            x_pos = (new_size - width) // 2

            # Crea un lienzo negro del tamaño objetivo
            canvas = np.zeros((new_size, new_size, 3), dtype=np.uint8)

            # Copia la imagen original en el lienzo negro en la posición calculada
            canvas[y_pos:y_pos+height, x_pos:x_pos+width, :] = img

            # Redimensiona la imagen resultante al tamaño final
            resized_img = cv2.resize(canvas, (final_size, final_size))
            
            # Guarda la imagen resultante en la carpeta de salida
            output_path = os.path.join(output_folder, filename)
            cv2.imwrite(output_path, resized_img)

# Define las carpetas de entrada y salida, y el tamaño objetivo
input_folder = 'tokio'
output_folder = 'tokioresize'
target_size = 512  # Puedes ajustar el tamaño deseado
final_size = 256
# Llama a la función para redimensionar y agregar márgenes a las imágenes
resize_and_add_margin(input_folder, output_folder, target_size, final_size)
